In [130]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [131]:
s = requests.Session()
s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'
url = 'http://www.bjsubway.com/e/action/ListInfo/?classid=39&ph=1'
response = s.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [132]:
#soup

In [172]:

station_dict = {}
for row in soup.select('table'):
    name = [x.text.strip() for x in row.find_all('td',{'colspan':['5','7','11','6','9']})][0].replace("（","").replace("）","")
    #print(sta_name)
    #name = sta_name
    sta_list = [re.findall('[\u4e00-\u9fff]+',x.text) for x in row.find_all('tbody')][0]
    
    #print(sta_list[0::2])
    if name[0] == '8':
        name = name[:5]
    elif name[:2] == '14':
        name = name[:5]
    elif name[:2] == '10':
        name = name[:3]
        sta_list = sta_list[0::2]
        sta_list.append('巴沟')
    elif name[:1] == '4':
        name = name[:3]
        sta_list = sta_list[0::2]
    elif name[:1] == '2':
        name = name[:3]
        sta_list.append('西直门')
        
    elif name[:1] == '6':
        name = name[:3]
        sta_list = list(filter(lambda a: a != '暂缓开通', sta_list))
    else:
        name = name[:3]
    station_dict[name] = sta_list
#print(len(station_dict))

21


In [229]:
#station_dict.keys()

In [228]:
#station_dict['4号线']

In [214]:
stations_line = defaultdict(set)
station_connection_info = defaultdict(list)
for line,v in station_dict.items():
    stations_line[v[0]].add(line)
    station_connection_info[v[0]].append(v[1])
    for i in range(1,len(v)-1):
        stations_line[v[i]].add(line)
        station_connection_info[v[i]].append(v[i-1])
        station_connection_info[v[i]].append(v[i+1])
    stations_line[v[len(v)-1]].add(line)
    station_connection_info[v[len(v)-1]].append(v[len(v)-2])    
    #print(k)
    #print(v)

In [227]:
#stations_line

In [216]:
def search(start, destination, connection_grpah, sort_candidate):
    
    pathes = [[start]]
    if start not in stations_line:
        print('"{}" No corresponding station'.format(start))
        return []
    if destination not in stations_line:
        print('"{}" No corresponding station'.format(destination))
    visited = set()
    while pathes:
        path = pathes.pop(0)#需要和sort_candidate一致
        #print(pathes)
        froninter = path[-1]
        
        if froninter in visited:continue
        successors = connection_grpah[froninter]
        for s in successors:
            
            if s in path:continue
            
            
            new_path = path+[s]
            #pathes.append(new_path)
            if new_path not in pathes:
                pathes.insert(len(pathes),new_path)
            if s == destination: return new_path
        visited.add(froninter)#'set' object has no attribute 'append'
        pathes = sort_candidate(pathes)
        #print(pathes)
    return pathes

In [270]:
confirm_station = defaultdict(dict)
def find_line(stations):
    global stations_line
    for i in range(len(stations)-1):
        confirm_station[stations[i]] = "".join(list(stations_line[stations[i]].intersection(stations_line[stations[i+1]])))
    confirm_station[stations[-1]] = confirm_station[stations[-2]]
    count = 0
    for i in range(1,len(stations)):
        
        if confirm_station[stations[i]] != confirm_station[stations[i-1]]:
            count += 1
            print("Change Station: {}".format(stations[i]))
            print("Change Line: {}".format("-".join([confirm_station[stations[i-1]],confirm_station[stations[i]]])))
    print("换乘次数: {}".format(count))
    return [stations[i]+'-'+str(confirm_station[stations[i]]) for i in range(len(stations))]

In [218]:
def pretty_print(cities):
    print('🚇->'.join(cities))

In [219]:
def transfer_line_as_less_possible(pathes):
    global stations_line
    if len(pathes) <= 1: return pathes
    def find_line2(path):
        confirm_station2 = set()
        for i in range(len(path)-1):
            confirm_station2.update(stations_line[path[i]].intersection(stations_line[path[i+1]]))

        return len(confirm_station2)

    return sorted(pathes, key=find_line2)

In [271]:
 pretty_print(find_line(search('劲松', '北京南站', station_connection_info, sort_candidate = lambda n:n)))

Change Station: 十里河
Change Line: 10号-14号线东
换乘次数: 1
劲松-10号🚇->潘家园-10号🚇->十里河-14号线东🚇->方庄-14号线东🚇->蒲黄榆-14号线东🚇->景泰-14号线东🚇->永定门外-14号线东🚇->北京南站-14号线东


In [272]:
 pretty_print(find_line(search('劲松', '将台', station_connection_info, sort_candidate = transfer_line_as_less_possible)))

Change Station: 呼家楼
Change Line: 10号-6号线
Change Station: 金台路
Change Line: 6号线-14号线东
换乘次数: 2
劲松-10号🚇->双井-10号🚇->国贸-10号🚇->金台夕照-10号🚇->呼家楼-6号线🚇->金台路-14号线东🚇->朝阳公园-14号线东🚇->枣营-14号线东🚇->东风北桥-14号线东🚇->将台-14号线东


In [273]:
 pretty_print(find_line(search('劲松', '将台', station_connection_info, sort_candidate = lambda n:n)))

Change Station: 国贸
Change Line: 10号-1号线
Change Station: 大望路
Change Line: 1号线-14号线东
换乘次数: 2
劲松-10号🚇->双井-10号🚇->国贸-1号线🚇->大望路-14号线东🚇->金台路-14号线东🚇->朝阳公园-14号线东🚇->枣营-14号线东🚇->东风北桥-14号线东🚇->将台-14号线东
